In [9]:
import pandas as pd
import vitalse_toolkit
import numpy as np
import pathlib
from pathlib import Path
import re
from typing import Union, Mapping
from functools import reduce
import time
from datetime import datetime

In [10]:
import pandas as pd
from rich.console import Console
from tqdm.auto import tqdm
import warnings
from vitalse_toolkit.sync import synchronizeFnirAndGazeData
from vitalse_toolkit.fixations import fixation_filters

console = Console()
gaze_file_pattern = r"gazeOutput-([0-9]+)\.xml"
itrace_file_pattern = r"itrace_core-([0-9]+)\.xml"
fnir_file_pattern = (
    r"^fS_Exported_oxygraph[0-9]\.ref\.(oxy|hbr|hbt|hbo)\.(Time|Block)[0-9]_")


def load_experiment_folder(root: Path):
    all_files = {
        "gazeOutput": None,
        "itraceOutput": None,
        "fnirFiles": {
            "hbr": {},
            "hbo": {},
            "oxy": {},
            "hbt": {},
            "log": {}
        },
    }
    console.log(f"Scanning folder [b]'{str(root)}'[/b] for experiment files")
    for file in root.iterdir():
        if file.suffix == ".log":
            all_files["fnirFiles"]["log"] = file
        elif len(matches := re.findall(gaze_file_pattern, file.name)) > 0:
            all_files["gazeOutput"] = file
        elif len(matches := re.findall(itrace_file_pattern, file.name)) > 0:
            all_files["itraceOutput"] = file
        elif len(matches := re.findall(fnir_file_pattern, file.name)) > 0:
            fnir_file_type, time_or_block = matches[0]
            all_files["fnirFiles"][fnir_file_type][
                time_or_block.lower()] = file
        else:
            continue

        console.log(f"> Found file [b]'{file.name}'[/b]")
    return all_files


FNIR_METRIC_KEYS = ["oxy", "hbr", "hbo", "hbt"]


def verify_files(all_files):
    assert "log" in all_files["fnirFiles"], "fNIR Log file not found"

    assert "itraceOutput" in all_files
    assert "gazeOutput" in all_files
    for k in FNIR_METRIC_KEYS:
        assert k in all_files["fnirFiles"], f"{k} fnir file not found"
        assert "time" in all_files["fnirFiles"][k]
        assert "block" in all_files["fnirFiles"][k]


def load_fnir_data(block_file: Union[str, Path], time_file: Union[str, Path]):
    optode_headers = ["optode{:02d}".format(num) for num in range(1, 19)]

    hbo = pd.read_csv(
        block_file,
        header=None,
        names=optode_headers,
        skiprows=17,
        na_values=["-"],
        skipinitialspace=True,
        index_col=False,
        encoding="latin1",
        # sep=None,
        delim_whitespace=True,
    )
    timestamps = pd.read_csv(time_file,
                             names=["Time"],
                             skiprows=17,
                             index_col=False,
                             encoding="latin1")

    fnir_data = pd.concat([timestamps, hbo], axis=1)
    fnir_data = fnir_data.dropna(axis=1, how="all")
    fnir_data = fnir_data.dropna(
        axis=0,
        subset=[col for col in fnir_data.columns if "optode" in col],
        how="all")
    return fnir_data


def load_fnir_files(files: Mapping[str, Mapping[str, Union[str, Path]]]):
    results = {}
    for metric in tqdm(FNIR_METRIC_KEYS):
        block = files[metric]["block"]
        time = files[metric]["time"]

        results[metric] = load_fnir_data(block, time)

    return results


def prep_fnir_df(df, metric, delay: float, start_time: float):
    # combine all fnir data into one
    # dataframe
    # cols_to_keep = getColsToKeep(raw_data["hbo"])
    cols_to_keep = [col for col in df.columns if "optode" in col]
    # cols_to_keep = raw_data['hbo'].drop('Time', axis=1).columns
    cols_dropped = 16 - len(cols_to_keep)
    colname = "{}_optode_avg".format(metric)
    fnir_df = df.copy()
    fnir_df[colname] = fnir_df.drop("Time", axis=1)[cols_to_keep].mean(axis=1)
    # print(fnir_df.to_markdown())
    fnir_df = fnir_df[["Time", colname]]

    # 1. Convert from seconds to milliseconds
    fnir_df["Time"] = fnir_df["Time"] * 1000

    # 2. Convert from time since experiment started
    #    to absolute time
    fnir_df["Time"] = fnir_df["Time"] + start_time

    # 3. Adjust for delay
    fnir_df["Time"] = fnir_df["Time"] - delay
    # fnir_df["optodes_dropped"] = cols_dropped

    return fnir_df


def extract_fnir_start_time(log_file: Union[str, Path]):
    # COBI Studio Modern Log File 2.2	1.2.7654.28485
    # Start Time:	Tue Nov 09 14:58:07.762 2021
    with open(log_file) as f:
        contents = f.read()
        pat = r"^Start Time:\s(.*)$"

        lines = contents.splitlines()
        assert "COBI Studio Modern Log File" in lines[0]
        matches = re.findall(pat, lines[1])
        assert len(matches) == 1, f"Log File not in correct format"
        date_string = matches[0]
        dt = datetime.strptime(date_string, "%a %b %d %H:%M:%S.%f %Y")

        return time.mktime(dt.timetuple()) * 1000


def prep_fnir_data(fnir_files, log_file, delay: float = 5000.0):
    dfs = []
    start_time = extract_fnir_start_time(log_file)
    for metric in fnir_files:
        df = prep_fnir_df(fnir_files[metric],
                          metric,
                          start_time=start_time,
                          delay=delay)
        dfs.append(df)

    return reduce(lambda a, b: pd.merge(a, b, how="inner", on="Time"), dfs)


Pathlike = Union[str, Path]


def process_eyetracking_data(core_file, plugin_file, time_col="plugin_time"):
    core_df = pd.read_xml(core_file, xpath=".//response")
    plugin_df = pd.read_xml(plugin_file, xpath=".//response")

    gazes = pd.merge(core_df,
                     plugin_df,
                     on=list(core_df.columns.intersection(plugin_df.columns)),
                     how='inner')
    console.log("FIXATIONS!!!")
    fixations = fixation_filters.ivt_gaze_classification(gazes, time=time_col)

    fixations = fixations.rename(
        {
            time_col: 'system_time',
            'gaze_target': 'target'
        }, axis=1)

    return fixations, gazes


def process_experiment_data(
    input_folder: Union[str, Path],
):
    if isinstance(input_folder, str):
        input_folder = Path(input_folder)

    all_files = load_experiment_folder(input_folder)

    verify_files(all_files)

    fixations, gazes = process_eyetracking_data(
        str(all_files["itraceOutput"]),
        str(all_files["gazeOutput"]),
    )

    valid_fixations = (fixations.source_file_line >=
                       0) & (fixations.source_file_col >= 0)
    n_valid_fixations = len(fixations[valid_fixations])
    n_total_fixations = len(fixations)
    console.log(
        f"Found {n_valid_fixations}/{n_total_fixations} ({n_valid_fixations/n_total_fixations * 100:.0f}%) valid fixations"
    )
    fixations = fixations[valid_fixations]

    fnir_files = load_fnir_files(all_files["fnirFiles"])
    fnir_df = prep_fnir_data(fnir_files, all_files["fnirFiles"]["log"])
    
    return fixations, fnir_df

In [16]:
experiment_folder = Path("/Users/devjeetroy/Library/CloudStorage/OneDrive-Personal/Control-Experiment-Spring-2022/p71/t1")

fixations, fnir_df = process_experiment_data(experiment_folder)

[12:10:59] Scanning folder '/Users/devjeetroy/Library/CloudStorage/OneDrive- ]8;id=567991;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=462413;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#27\27]8;;\
           Personal/Control-Experiment-Spring-2022/p71/t1' for experiment                    
           files                                                                             

           > Found file 'gazeOutput-1668458592202.xml'                       ]8;id=448853;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=18329;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#42\42]8;;\

           > Found file 'itrace_core-1668458592202.xml'                      ]8;id=495049;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=182126;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#42\42]8;;\

           > Found file                                                      ]8;id=972571;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=633319;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#42\42]8;;\
           'fS_Exported_oxygraph1.ref.hbr.Block1_2022-11-14.txt'                             

           > Found file                                                      ]8;id=320081;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=794770;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#42\42]8;;\
           'fS_Exported_oxygraph1.ref.oxy.Block1_2022-11-14.txt'                             

...

           > Found file 'fS_Exported_oxygraph1.ref.hbo.Time1_2022-11-14.txt' ]8;id=771242;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=730249;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#42\42]8;;\

           > Found file 'fS_Exported_oxygraph1.ref.hbr.Time1_2022-11-14.txt' ]8;id=353913;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=986261;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#42\42]8;;\

[12:11:03] FIXATIONS!!!                                                     ]8;id=209414;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=567622;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#169\169]8;;\

           Found 375/375 (100%) valid fixations                             ]8;id=190636;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py\4133005103.py]8;;\:]8;id=758304;file:///var/folders/9t/pnh1mqks36vfw7ngs3d7n8r00000gn/T/ipykernel_99292/4133005103.py#200\200]8;;\

100%|██████████| 4/4 [00:00<00:00, 80.81it/s]


In [13]:
fnir_df['Time'] = fnir_df['Time'].astype(int)

fnir_df

,Time,oxy_optode_avg,hbr_optode_avg,hbo_optode_avg,hbt_optode_avg
0,1668458511547,0.640909,-0.174048,0.466861,0.292813
1,1668458511647,0.686689,-0.194216,0.492473,0.298257
2,1668458511747,0.729879,-0.213539,0.516340,0.302802
3,1668458511847,0.770118,-0.231848,0.538271,0.306423
4,1668458511947,0.807064,-0.248986,0.558078,0.309091
...,...,...,...,...,...
3016,1668458813147,6.101099,-2.236292,3.864807,1.628514
3017,1668458813247,6.086905,-2.228169,3.858735,1.630566
3018,1668458813347,6.072560,-2.219853,3.852707,1.632855
3019,1668458813447,6.058098,-2.211355,3.846743,1.635388


In [14]:
sync = synchronizeFnirAndGazeData(fixations, fnir_df)

processing gazes:  98%|█████████▊| 366/375 [00:00<00:00, 1113.01it/s]


In [15]:
sync

,duration,n,system_time,source_file_line,source_file_col,target,left_pupil_diameter,right_pupil_diameter,x,y,oxy_optode_avg,hbr_optode_avg,hbo_optode_avg,hbt_optode_avg
0,184.0,24.0,1.668459e+12,563.0,331.0,numbers_c.cc,19.956520,20.055902,559.958333,315.250000,1.039076,-0.501780,0.537296,0.035516
1,139.0,19.0,1.668459e+12,1479.0,152.0,numbers_c.cc,19.084887,17.165685,1483.368421,175.263158,1.123727,-0.540236,0.583490,0.043254
2,245.0,29.0,1.668459e+12,1532.0,164.0,numbers_c.cc,17.251362,15.748727,1525.517241,227.103448,1.202072,-0.575994,0.626078,0.050083
3,87.0,11.0,1.668459e+12,1536.0,263.0,numbers_c.cc,16.460735,14.674297,1559.818182,244.727273,1.258257,-0.601618,0.656639,0.055021
4,120.0,14.0,1.668459e+12,563.0,319.0,numbers_c.cc,17.511614,18.457665,575.142857,328.857143,1.304663,-0.622620,0.682043,0.059423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,108.0,14.0,1.668459e+12,572.0,814.0,numbers_c.cc,19.933499,19.996755,542.785714,809.571429,6.176638,-2.277725,3.898913,1.621188
362,125.0,15.0,1.668459e+12,565.0,924.0,numbers_c.cc,18.921117,19.894693,560.200000,935.533333,6.156090,-2.266760,3.889330,1.622570
363,374.0,47.0,1.668459e+12,529.0,490.0,numbers_c.cc,21.096839,21.639098,564.021277,500.765957,6.121891,-2.247936,3.873955,1.626018
364,164.0,18.0,1.668459e+12,599.0,402.0,numbers_c.cc,20.713243,21.434397,589.666667,399.611111,6.079732,-2.224011,3.855721,1.631711


In [17]:
df = pd.read_json(experiment_folder / "processed-new/sync.json")

In [18]:
df

,duration,n,system_time,source_file_line,source_file_col,target,left_pupil_diameter,right_pupil_diameter,x,y,oxy_optode_avg,hbr_optode_avg,hbo_optode_avg,hbt_optode_avg
0,184,24,2022-11-14 20:43:12.358000128,563,331,numbers_c.cc,19.956520,20.055902,559.958333,315.250000,1.039076,-0.501780,0.537296,0.035516
1,139,19,2022-11-14 20:43:12.619000064,1479,152,numbers_c.cc,19.084887,17.165685,1483.368421,175.263158,1.123727,-0.540236,0.583490,0.043254
2,245,29,2022-11-14 20:43:12.772999936,1532,164,numbers_c.cc,17.251362,15.748727,1525.517241,227.103448,1.202072,-0.575994,0.626078,0.050083
3,87,11,2022-11-14 20:43:13.087000064,1536,263,numbers_c.cc,16.460735,14.674297,1559.818182,244.727273,1.258257,-0.601618,0.656639,0.055021
4,120,14,2022-11-14 20:43:13.268000000,563,319,numbers_c.cc,17.511614,18.457665,575.142857,328.857143,1.304663,-0.622620,0.682043,0.059423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,108,14,2022-11-14 20:46:52.479000064,572,814,numbers_c.cc,19.933499,19.996755,542.785714,809.571429,6.176638,-2.277725,3.898913,1.621188
362,125,15,2022-11-14 20:46:52.601999872,565,924,numbers_c.cc,18.921117,19.894693,560.200000,935.533333,6.156090,-2.266760,3.889330,1.622570
363,374,47,2022-11-14 20:46:52.772999936,529,490,numbers_c.cc,21.096839,21.639098,564.021277,500.765957,6.121891,-2.247936,3.873955,1.626018
364,164,18,2022-11-14 20:46:53.191000064,599,402,numbers_c.cc,20.713243,21.434397,589.666667,399.611111,6.079732,-2.224011,3.855721,1.631711
